In [ ]:
import math

# --- GLOBAL CONSTANTS (Typical values for air and fuel) ---
GAMMA_AIR = 1.4
CP_AIR = 1005
R_AIR = 287
HEATING_VALUE_FUEL = 43.0e6

# --- HELPER FUNCTIONS ---

def CALCULATE_ISENTROPIC_TEMPERATURE_RATIO(PRESSURE_RATIO, GAMMA):
    """Calculates isentropic temperature ratio for a given pressure ratio."""
    return (PRESSURE_RATIO)**((GAMMA - 1) / GAMMA)

def CALCULATE_ISENTROPIC_PRESSURE_RATIO(TEMPERATURE_RATIO, GAMMA):
    """Calculates isentropic pressure ratio for a given temperature ratio."""
    return (TEMPERATURE_RATIO)**(GAMMA / (GAMMA - 1))

# --- ENGINE PERFORMANCE ESTIMATORS ---

def ESTIMATE_TURBOJET_PERFORMANCE(
    FLIGHT_MACH_NUMBER,
    AMBIENT_TEMPERATURE_K,
    AMBIENT_PRESSURE_PA,
    COMPRESSOR_PRESSURE_RATIO,
    TURBINE_INLET_TEMPERATURE_K,
    COMPRESSOR_ISENTROPIC_EFFICIENCY=1.0,
    TURBINE_ISENTROPIC_EFFICIENCY=1.0,
    NOZZLE_ISENTROPIC_EFFICIENCY=1.0,
    DIFFUSER_ISENTROPIC_EFFICIENCY=1.0
):
    """
    Estimates performance for an ideal turbojet engine.
    Assumes constant pressure combustion and negligible pressure losses in combustor.
    """
    print("\n--- Turbojet Engine Analysis ---")

    VELOCITY_FLIGHT = FLIGHT_MACH_NUMBER * math.sqrt(GAMMA_AIR * R_AIR * AMBIENT_TEMPERATURE_K)

    TOTAL_TEMPERATURE_2 = AMBIENT_TEMPERATURE_K * (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)
    TOTAL_PRESSURE_2_IDEAL_RATIO = (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)**(GAMMA_AIR / (GAMMA_AIR - 1))
    TOTAL_PRESSURE_2 = AMBIENT_PRESSURE_PA * (1 + DIFFUSER_ISENTROPIC_EFFICIENCY * (TOTAL_PRESSURE_2_IDEAL_RATIO - 1))

    TOTAL_PRESSURE_3 = TOTAL_PRESSURE_2 * COMPRESSOR_PRESSURE_RATIO
    TEMPERATURE_RATIO_COMPRESSOR_ISENTROPIC = CALCULATE_ISENTROPIC_TEMPERATURE_RATIO(COMPRESSOR_PRESSURE_RATIO, GAMMA_AIR)
    TOTAL_TEMPERATURE_3 = TOTAL_TEMPERATURE_2 * (1 + (TEMPERATURE_RATIO_COMPRESSOR_ISENTROPIC - 1) / COMPRESSOR_ISENTROPIC_EFFICIENCY)

    COMPRESSOR_WORK = CP_AIR * (TOTAL_TEMPERATURE_3 - TOTAL_TEMPERATURE_2)

    TOTAL_PRESSURE_4 = TOTAL_PRESSURE_3
    FUEL_AIR_RATIO = (CP_AIR * (TURBINE_INLET_TEMPERATURE_K - TOTAL_TEMPERATURE_3)) / (HEATING_VALUE_FUEL - CP_AIR * TURBINE_INLET_TEMPERATURE_K)
    if FUEL_AIR_RATIO < 0:
        print("Error: Turbine Inlet Temperature (T_t4) is too low for combustion given T_t3.")
        return None, None

    TOTAL_TEMPERATURE_5 = TURBINE_INLET_TEMPERATURE_K - (COMPRESSOR_WORK / CP_AIR)
    if TOTAL_TEMPERATURE_5 <= 0:
        print("Error: Turbine Inlet Temperature (T_t4) is too low to drive the compressor.")
        return None, None

    TEMPERATURE_RATIO_TURBINE_ISENTROPIC = 1 - (1 - TOTAL_TEMPERATURE_5 / TURBINE_INLET_TEMPERATURE_K) / TURBINE_ISENTROPIC_EFFICIENCY
    TURBINE_PRESSURE_RATIO = CALCULATE_ISENTROPIC_PRESSURE_RATIO(1 / TEMPERATURE_RATIO_TURBINE_ISENTROPIC, GAMMA_AIR)
    TOTAL_PRESSURE_5 = TOTAL_PRESSURE_4 / TURBINE_PRESSURE_RATIO

    if TOTAL_PRESSURE_5 < AMBIENT_PRESSURE_PA:
        print("Warning: Nozzle exit pressure (P_t5) is less than ambient pressure. Thrust calculation might be inaccurate.")
        STATIC_TEMPERATURE_9_IDEAL = TOTAL_TEMPERATURE_5 * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_5)**((GAMMA_AIR - 1) / GAMMA_AIR)
    else:
        STATIC_TEMPERATURE_9_IDEAL = TOTAL_TEMPERATURE_5 * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_5)**((GAMMA_AIR - 1) / GAMMA_AIR)

    ACTUAL_TEMPERATURE_DROP_NOZZLE = NOZZLE_ISENTROPIC_EFFICIENCY * (TOTAL_TEMPERATURE_5 - STATIC_TEMPERATURE_9_IDEAL)
    STATIC_TEMPERATURE_9_ACTUAL = TOTAL_TEMPERATURE_5 - ACTUAL_TEMPERATURE_DROP_NOZZLE

    print(f"Debug: TOTAL_TEMPERATURE_5 = {TOTAL_TEMPERATURE_5}")
    print(f"Debug: STATIC_TEMPERATURE_9_ACTUAL = {STATIC_TEMPERATURE_9_ACTUAL}")

    VELOCITY_EXIT = math.sqrt(2 * CP_AIR * (TOTAL_TEMPERATURE_5 - STATIC_TEMPERATURE_9_ACTUAL))
    if math.isnan(VELOCITY_EXIT) or VELOCITY_EXIT < 0:
        print("Error: Invalid exit velocity calculated. Check input parameters.")
        return None, None

    NET_THRUST_PER_UNIT_MASS_FLOW = (1 + FUEL_AIR_RATIO) * VELOCITY_EXIT - VELOCITY_FLIGHT

    if NET_THRUST_PER_UNIT_MASS_FLOW <= 0:
        SPECIFIC_FUEL_CONSUMPTION = float('inf')
        print("Warning: Net Thrust is zero or negative. Check input parameters.")
    else:
        SPECIFIC_FUEL_CONSUMPTION = FUEL_AIR_RATIO / NET_THRUST_PER_UNIT_MASS_FLOW

    print(f"  Flight Mach Number: {FLIGHT_MACH_NUMBER:.2f}")
    print(f"  Compressor Pressure Ratio: {COMPRESSOR_PRESSURE_RATIO:.2f}")
    print(f"  Turbine Inlet Temperature: {TURBINE_INLET_TEMPERATURE_K:.0f} K")
    print(f"  Fuel-Air Ratio (f): {FUEL_AIR_RATIO:.4f}")
    print(f"  Net Thrust per unit Air Mass Flow: {NET_THRUST_PER_UNIT_MASS_FLOW:.2f} N/(kg/s)")
    print(f"  Thrust Specific Fuel Consumption (TSFC): {SPECIFIC_FUEL_CONSUMPTION*1e6:.2f} mg/(N·s) or {SPECIFIC_FUEL_CONSUMPTION*3600:.4f} kg/(kN·hr)")

    return NET_THRUST_PER_UNIT_MASS_FLOW, SPECIFIC_FUEL_CONSUMPTION

def ESTIMATE_TURBOFAN_PERFORMANCE(
    FLIGHT_MACH_NUMBER,
    AMBIENT_TEMPERATURE_K,
    AMBIENT_PRESSURE_PA,
    COMPRESSOR_PRESSURE_RATIO,
    FAN_PRESSURE_RATIO,
    BYPASS_RATIO,
    TURBINE_INLET_TEMPERATURE_K,
    COMPRESSOR_ISENTROPIC_EFFICIENCY=1.0,
    TURBINE_ISENTROPIC_EFFICIENCY=1.0,
    FAN_ISENTROPIC_EFFICIENCY=1.0,
    NOZZLE_ISENTROPIC_EFFICIENCY=1.0,
    DIFFUSER_ISENTROPIC_EFFICIENCY=1.0
):
    """
    Estimates performance for an ideal turbofan engine (two-spool, separate nozzles).
    Assumes constant pressure combustion and negligible pressure losses in combustor.
    """
    print("\n--- Turbofan Engine Analysis ---")

    VELOCITY_FLIGHT = FLIGHT_MACH_NUMBER * math.sqrt(GAMMA_AIR * R_AIR * AMBIENT_TEMPERATURE_K)

    TOTAL_TEMPERATURE_2 = AMBIENT_TEMPERATURE_K * (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)
    TOTAL_PRESSURE_2_IDEAL_RATIO = (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)**(GAMMA_AIR / (GAMMA_AIR - 1))
    TOTAL_PRESSURE_2 = AMBIENT_PRESSURE_PA * (1 + DIFFUSER_ISENTROPIC_EFFICIENCY * (TOTAL_PRESSURE_2_IDEAL_RATIO - 1))

    FAN_WORK_PER_UNIT_FAN_MASS_FLOW = CP_AIR * (TOTAL_TEMPERATURE_2 * (1 + (CALCULATE_ISENTROPIC_TEMPERATURE_RATIO(FAN_PRESSURE_RATIO, GAMMA_AIR) - 1) / FAN_ISENTROPIC_EFFICIENCY) - TOTAL_TEMPERATURE_2)
    TOTAL_TEMPERATURE_2_5_CORE = TOTAL_TEMPERATURE_2 * (1 + (CALCULATE_ISENTROPIC_TEMPERATURE_RATIO(FAN_PRESSURE_RATIO, GAMMA_AIR) - 1) / FAN_ISENTROPIC_EFFICIENCY)
    TOTAL_PRESSURE_2_5_CORE = TOTAL_PRESSURE_2 * FAN_PRESSURE_RATIO

    TOTAL_PRESSURE_3_CORE = TOTAL_PRESSURE_2_5_CORE * COMPRESSOR_PRESSURE_RATIO
    TEMPERATURE_RATIO_COMPRESSOR_ISENTROPIC = CALCULATE_ISENTROPIC_TEMPERATURE_RATIO(COMPRESSOR_PRESSURE_RATIO, GAMMA_AIR)
    TOTAL_TEMPERATURE_3_CORE = TOTAL_TEMPERATURE_2_5_CORE * (1 + (TEMPERATURE_RATIO_COMPRESSOR_ISENTROPIC - 1) / COMPRESSOR_ISENTROPIC_EFFICIENCY)

    CORE_COMPRESSOR_WORK = CP_AIR * (TOTAL_TEMPERATURE_3_CORE - TOTAL_TEMPERATURE_2_5_CORE)

    TOTAL_PRESSURE_4_CORE = TOTAL_PRESSURE_3_CORE
    FUEL_AIR_RATIO_CORE = (CP_AIR * (TURBINE_INLET_TEMPERATURE_K - TOTAL_TEMPERATURE_3_CORE)) / (HEATING_VALUE_FUEL - CP_AIR * TURBINE_INLET_TEMPERATURE_K)
    if FUEL_AIR_RATIO_CORE < 0:
        print("Error: Turbine Inlet Temperature (T_t4) is too low for combustion in core.")
        return None, None

    TOTAL_TURBINE_WORK_PER_UNIT_CORE_MASS_FLOW = CORE_COMPRESSOR_WORK + BYPASS_RATIO * FAN_WORK_PER_UNIT_FAN_MASS_FLOW

    TOTAL_TEMPERATURE_5_CORE = TURBINE_INLET_TEMPERATURE_K - (TOTAL_TURBINE_WORK_PER_UNIT_CORE_MASS_FLOW / CP_AIR)
    if TOTAL_TEMPERATURE_5_CORE <= 0:
        print("Error: Turbine Inlet Temperature (T_t4) is too low to drive compressor and fan.")
        return None, None

    TEMPERATURE_RATIO_TURBINE_ISENTROPIC = 1 - (1 - TOTAL_TEMPERATURE_5_CORE / TURBINE_INLET_TEMPERATURE_K) / TURBINE_ISENTROPIC_EFFICIENCY
    TURBINE_PRESSURE_RATIO = CALCULATE_ISENTROPIC_PRESSURE_RATIO(1 / TEMPERATURE_RATIO_TURBINE_ISENTROPIC, GAMMA_AIR)
    TOTAL_PRESSURE_5_CORE = TOTAL_PRESSURE_4_CORE / TURBINE_PRESSURE_RATIO

    STATIC_TEMPERATURE_9_CORE_IDEAL = TOTAL_TEMPERATURE_5_CORE * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_5_CORE)**((GAMMA_AIR - 1) / GAMMA_AIR)
    ACTUAL_TEMPERATURE_DROP_CORE_NOZZLE = NOZZLE_ISENTROPIC_EFFICIENCY * (TOTAL_TEMPERATURE_5_CORE - STATIC_TEMPERATURE_9_CORE_IDEAL)
    STATIC_TEMPERATURE_9_CORE_ACTUAL = TOTAL_TEMPERATURE_5_CORE - ACTUAL_TEMPERATURE_DROP_CORE_NOZZLE
    VELOCITY_EXIT_CORE = math.sqrt(2 * CP_AIR * (TOTAL_TEMPERATURE_5_CORE - STATIC_TEMPERATURE_9_CORE_ACTUAL))
    if math.isnan(VELOCITY_EXIT_CORE) or VELOCITY_EXIT_CORE < 0:
        print("Error: Invalid core exit velocity calculated.")
        return None, None

    CORE_THRUST_PER_UNIT_CORE_MASS_FLOW = (1 + FUEL_AIR_RATIO_CORE) * VELOCITY_EXIT_CORE - VELOCITY_FLIGHT

    STATIC_TEMPERATURE_19_BYPASS_IDEAL = TOTAL_TEMPERATURE_2_5_CORE * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_2_5_CORE)**((GAMMA_AIR - 1) / GAMMA_AIR)
    ACTUAL_TEMPERATURE_DROP_BYPASS_NOZZLE = NOZZLE_ISENTROPIC_EFFICIENCY * (TOTAL_TEMPERATURE_2_5_CORE - STATIC_TEMPERATURE_19_BYPASS_IDEAL)
    STATIC_TEMPERATURE_19_BYPASS_ACTUAL = TOTAL_TEMPERATURE_2_5_CORE - ACTUAL_TEMPERATURE_DROP_BYPASS_NOZZLE
    VELOCITY_EXIT_BYPASS = math.sqrt(2 * CP_AIR * (TOTAL_TEMPERATURE_2_5_CORE - STATIC_TEMPERATURE_19_BYPASS_ACTUAL))
    if math.isnan(VELOCITY_EXIT_BYPASS) or VELOCITY_EXIT_BYPASS < 0:
        print("Error: Invalid bypass exit velocity calculated.")
        return None, None

    BYPASS_THRUST_PER_UNIT_BYPASS_MASS_FLOW = VELOCITY_EXIT_BYPASS - VELOCITY_FLIGHT

    TOTAL_NET_THRUST_PER_UNIT_INLET_MASS_FLOW = CORE_THRUST_PER_UNIT_CORE_MASS_FLOW + BYPASS_RATIO * BYPASS_THRUST_PER_UNIT_BYPASS_MASS_FLOW

    TOTAL_FUEL_AIR_RATIO = FUEL_AIR_RATIO_CORE
    if TOTAL_NET_THRUST_PER_UNIT_INLET_MASS_FLOW <= 0:
        SPECIFIC_FUEL_CONSUMPTION = float('inf')
        print("Warning: Total Net Thrust is zero or negative. Check input parameters.")
    else:
        SPECIFIC_FUEL_CONSUMPTION = TOTAL_FUEL_AIR_RATIO / TOTAL_NET_THRUST_PER_UNIT_INLET_MASS_FLOW

    print(f"  Flight Mach Number: {FLIGHT_MACH_NUMBER:.2f}")
    print(f"  Compressor Pressure Ratio: {COMPRESSOR_PRESSURE_RATIO:.2f}")
    print(f"  Fan Pressure Ratio: {FAN_PRESSURE_RATIO:.2f}")
    print(f"  Bypass Ratio: {BYPASS_RATIO:.1f}")
    print(f"  Turbine Inlet Temperature: {TURBINE_INLET_TEMPERATURE_K:.0f} K")
    print(f"  Core Fuel-Air Ratio (f): {FUEL_AIR_RATIO_CORE:.4f}")
    print(f"  Total Net Thrust per unit Inlet Air Mass Flow: {TOTAL_NET_THRUST_PER_UNIT_INLET_MASS_FLOW:.2f} N/(kg/s)")
    print(f"  Thrust Specific Fuel Consumption (TSFC): {SPECIFIC_FUEL_CONSUMPTION*1e6:.2f} mg/(N·s) or {SPECIFIC_FUEL_CONSUMPTION*3600:.4f} kg/(kN·hr)")

    return TOTAL_NET_THRUST_PER_UNIT_INLET_MASS_FLOW, SPECIFIC_FUEL_CONSUMPTION

def ESTIMATE_TURBOPROP_PERFORMANCE(
    FLIGHT_MACH_NUMBER,
    AMBIENT_TEMPERATURE_K,
    AMBIENT_PRESSURE_PA,
    COMPRESSOR_PRESSURE_RATIO,
    TURBINE_INLET_TEMPERATURE_K,
    PROPELLER_EFFICIENCY,
    COMPRESSOR_ISENTROPIC_EFFICIENCY=1.0,
    TURBINE_ISENTROPIC_EFFICIENCY=1.0,
    NOZZLE_ISENTROPIC_EFFICIENCY=1.0,
    DIFFUSER_ISENTROPIC_EFFICIENCY=1.0
):
    """
    Estimates performance for an ideal turboprop engine.
    Focuses on shaft power output, which drives the propeller.
    Assumes all net turbine work goes to shaft power after driving compressor.
    Small residual jet thrust is also calculated.
    """
    print("\n--- Turboprop Engine Analysis ---")

    VELOCITY_FLIGHT = FLIGHT_MACH_NUMBER * math.sqrt(GAMMA_AIR * R_AIR * AMBIENT_TEMPERATURE_K)

    TOTAL_TEMPERATURE_2 = AMBIENT_TEMPERATURE_K * (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)
    TOTAL_PRESSURE_2_IDEAL_RATIO = (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)**(GAMMA_AIR / (GAMMA_AIR - 1))
    TOTAL_PRESSURE_2 = AMBIENT_PRESSURE_PA * (1 + DIFFUSER_ISENTROPIC_EFFICIENCY * (TOTAL_PRESSURE_2_IDEAL_RATIO - 1))

    TOTAL_PRESSURE_3 = TOTAL_PRESSURE_2 * COMPRESSOR_PRESSURE_RATIO
    TEMPERATURE_RATIO_COMPRESSOR_ISENTROPIC = CALCULATE_ISENTROPIC_TEMPERATURE_RATIO(COMPRESSOR_PRESSURE_RATIO, GAMMA_AIR)
    TOTAL_TEMPERATURE_3 = TOTAL_TEMPERATURE_2 * (1 + (TEMPERATURE_RATIO_COMPRESSOR_ISENTROPIC - 1) / COMPRESSOR_ISENTROPIC_EFFICIENCY)
    COMPRESSOR_WORK = CP_AIR * (TOTAL_TEMPERATURE_3 - TOTAL_TEMPERATURE_2)

    TOTAL_PRESSURE_4 = TOTAL_PRESSURE_3
    FUEL_AIR_RATIO = (CP_AIR * (TURBINE_INLET_TEMPERATURE_K - TOTAL_TEMPERATURE_3)) / (HEATING_VALUE_FUEL - CP_AIR * TURBINE_INLET_TEMPERATURE_K)
    if FUEL_AIR_RATIO < 0:
        print("Error: Turbine Inlet Temperature (T_t4) is too low for combustion.")
        return None, None

    TOTAL_TEMPERATURE_5_IDEAL_TURBINE_EXIT = TURBINE_INLET_TEMPERATURE_K * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_4)**((GAMMA_AIR - 1) / GAMMA_AIR)
    TOTAL_TEMPERATURE_5_ACTUAL_TURBINE_EXIT = TURBINE_INLET_TEMPERATURE_K - TURBINE_ISENTROPIC_EFFICIENCY * (TURBINE_INLET_TEMPERATURE_K - TOTAL_TEMPERATURE_5_IDEAL_TURBINE_EXIT)

    TOTAL_TURBINE_WORK = CP_AIR * (TURBINE_INLET_TEMPERATURE_K - TOTAL_TEMPERATURE_5_ACTUAL_TURBINE_EXIT)

    SHAFT_WORK_PER_UNIT_MASS_FLOW = TOTAL_TURBINE_WORK - COMPRESSOR_WORK
    if SHAFT_WORK_PER_UNIT_MASS_FLOW < 0:
        print("Error: Not enough turbine work to drive compressor and produce shaft power.")
        return None, None

    PRESSURE_RATIO_TURBINE_ACTUAL = CALCULATE_ISENTROPIC_PRESSURE_RATIO(TURBINE_INLET_TEMPERATURE_K / (TURBINE_INLET_TEMPERATURE_K - (TOTAL_TURBINE_WORK / CP_AIR)), GAMMA_AIR)
    TOTAL_PRESSURE_5 = TOTAL_PRESSURE_4 / PRESSURE_RATIO_TURBINE_ACTUAL

    STATIC_TEMPERATURE_9_JET_IDEAL = TOTAL_TEMPERATURE_5_ACTUAL_TURBINE_EXIT * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_5)**((GAMMA_AIR - 1) / GAMMA_AIR)
    ACTUAL_TEMPERATURE_DROP_JET_NOZZLE = NOZZLE_ISENTROPIC_EFFICIENCY * (TOTAL_TEMPERATURE_5_ACTUAL_TURBINE_EXIT - STATIC_TEMPERATURE_9_JET_IDEAL)
    STATIC_TEMPERATURE_9_JET_ACTUAL = TOTAL_TEMPERATURE_5_ACTUAL_TURBINE_EXIT - ACTUAL_TEMPERATURE_DROP_JET_NOZZLE
    VELOCITY_EXIT_JET = math.sqrt(2 * CP_AIR * (TOTAL_TEMPERATURE_5_ACTUAL_TURBINE_EXIT - STATIC_TEMPERATURE_9_JET_ACTUAL))
    if math.isnan(VELOCITY_EXIT_JET) or VELOCITY_EXIT_JET < 0:
        print("Error: Invalid jet exit velocity calculated for turboprop.")
        return None, None

    JET_THRUST_PER_UNIT_MASS_FLOW = (1 + FUEL_AIR_RATIO) * VELOCITY_EXIT_JET - VELOCITY_FLIGHT

    if VELOCITY_FLIGHT == 0:
        PROPELLER_THRUST_PER_UNIT_MASS_FLOW = float('inf')
        print("Warning: Flight velocity is zero. Propeller thrust calculation is simplified.")
    else:
        PROPELLER_THRUST_PER_UNIT_MASS_FLOW = SHAFT_WORK_PER_UNIT_MASS_FLOW * PROPELLER_EFFICIENCY / VELOCITY_FLIGHT

    TOTAL_NET_THRUST_PER_UNIT_MASS_FLOW = PROPELLER_THRUST_PER_UNIT_MASS_FLOW + JET_THRUST_PER_UNIT_MASS_FLOW

    if TOTAL_NET_THRUST_PER_UNIT_MASS_FLOW <= 0:
        SPECIFIC_FUEL_CONSUMPTION = float('inf')
        print("Warning: Total Net Thrust is zero or negative. Check input parameters.")
    else:
        SPECIFIC_FUEL_CONSUMPTION = FUEL_AIR_RATIO / TOTAL_NET_THRUST_PER_UNIT_MASS_FLOW

    print(f"  Flight Mach Number: {FLIGHT_MACH_NUMBER:.2f}")
    print(f"  Compressor Pressure Ratio: {COMPRESSOR_PRESSURE_RATIO:.2f}")
    print(f"  Turbine Inlet Temperature: {TURBINE_INLET_TEMPERATURE_K:.0f} K")
    print(f"  Propeller Efficiency: {PROPELLER_EFFICIENCY:.2f}")
    print(f"  Fuel-Air Ratio (f): {FUEL_AIR_RATIO:.4f}")
    print(f"  Shaft Work per unit Air Mass Flow: {SHAFT_WORK_PER_UNIT_MASS_FLOW:.2f} J/kg")
    print(f"  Propeller Thrust per unit Air Mass Flow: {PROPELLER_THRUST_PER_UNIT_MASS_FLOW:.2f} N/(kg/s)")
    print(f"  Jet Thrust per unit Air Mass Flow: {JET_THRUST_PER_UNIT_MASS_FLOW:.2f} N/(kg/s)")
    print(f"  Total Net Thrust per unit Inlet Air Mass Flow: {TOTAL_NET_THRUST_PER_UNIT_MASS_FLOW:.2f} N/(kg/s)")
    print(f"  Thrust Specific Fuel Consumption (TSFC): {SPECIFIC_FUEL_CONSUMPTION*1e6:.2f} mg/(N·s) or {SPECIFIC_FUEL_CONSUMPTION*3600:.4f} kg/(kN·hr)")

    return TOTAL_NET_THRUST_PER_UNIT_MASS_FLOW, SPECIFIC_FUEL_CONSUMPTION

def ESTIMATE_RAMJET_PERFORMANCE(
    FLIGHT_MACH_NUMBER,
    AMBIENT_TEMPERATURE_K,
    AMBIENT_PRESSURE_PA,
    COMBUSTOR_OUTLET_TOTAL_TEMPERATURE_K,
    DIFFUSER_ISENTROPIC_EFFICIENCY=1.0,
    NOZZLE_ISENTROPIC_EFFICIENCY=1.0
):
    """
    Estimates performance for an ideal ramjet engine.
    Ramjets require supersonic flight Mach numbers to operate efficiently.
    No compressor or turbine.
    """
    print("\n--- Ramjet Engine Analysis ---")

    if FLIGHT_MACH_NUMBER < 1.0:
        print("Warning: Ramjets are typically designed for supersonic flight (Mach > 1.0).")
        print("Performance estimates at subsonic speeds will be very poor or zero.")

    VELOCITY_FLIGHT = FLIGHT_MACH_NUMBER * math.sqrt(GAMMA_AIR * R_AIR * AMBIENT_TEMPERATURE_K)

    TOTAL_TEMPERATURE_2 = AMBIENT_TEMPERATURE_K * (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)
    TOTAL_PRESSURE_2_IDEAL_RATIO = (1 + (GAMMA_AIR - 1) / 2 * FLIGHT_MACH_NUMBER**2)**(GAMMA_AIR / (GAMMA_AIR - 1))
    TOTAL_PRESSURE_2 = AMBIENT_PRESSURE_PA * (1 + DIFFUSER_ISENTROPIC_EFFICIENCY * (TOTAL_PRESSURE_2_IDEAL_RATIO - 1))

    TOTAL_TEMPERATURE_3 = COMBUSTOR_OUTLET_TOTAL_TEMPERATURE_K
    TOTAL_PRESSURE_3 = TOTAL_PRESSURE_2

    FUEL_AIR_RATIO = (CP_AIR * (TOTAL_TEMPERATURE_3 - TOTAL_TEMPERATURE_2)) / (HEATING_VALUE_FUEL - CP_AIR * TOTAL_TEMPERATURE_3)
    if FUEL_AIR_RATIO < 0:
        print("Error: Combustor Outlet Temperature (T_t3) is too low for combustion given T_t2.")
        return None, None

    STATIC_TEMPERATURE_4_IDEAL = TOTAL_TEMPERATURE_3 * (AMBIENT_PRESSURE_PA / TOTAL_PRESSURE_3)**((GAMMA_AIR - 1) / GAMMA_AIR)
    ACTUAL_TEMPERATURE_DROP_NOZZLE = NOZZLE_ISENTROPIC_EFFICIENCY * (TOTAL_TEMPERATURE_3 - STATIC_TEMPERATURE_4_IDEAL)
    STATIC_TEMPERATURE_4_ACTUAL = TOTAL_TEMPERATURE_3 - ACTUAL_TEMPERATURE_DROP_NOZZLE
    VELOCITY_EXIT = math.sqrt(2 * CP_AIR * (TOTAL_TEMPERATURE_3 - STATIC_TEMPERATURE_4_ACTUAL))
    if math.isnan(VELOCITY_EXIT) or VELOCITY_EXIT < 0:
        print("Error: Invalid exit velocity calculated. Check input parameters.")
        return None, None

    NET_THRUST_PER_UNIT_MASS_FLOW = (1 + FUEL_AIR_RATIO) * VELOCITY_EXIT - VELOCITY_FLIGHT

    if NET_THRUST_PER_UNIT_MASS_FLOW <= 0:
        SPECIFIC_FUEL_CONSUMPTION = float('inf')
        print("Warning: Net Thrust is zero or negative. Check input parameters.")
    else:
        SPECIFIC_FUEL_CONSUMPTION = FUEL_AIR_RATIO / NET_THRUST_PER_UNIT_MASS_FLOW

    print(f"  Flight Mach Number: {FLIGHT_MACH_NUMBER:.2f}")
    print(f"  Combustor Outlet Temperature: {COMBUSTOR_OUTLET_TOTAL_TEMPERATURE_K:.0f} K")
    print(f"  Fuel-Air Ratio (f): {FUEL_AIR_RATIO:.4f}")
    print(f"  Net Thrust per unit Air Mass Flow: {NET_THRUST_PER_UNIT_MASS_FLOW:.2f} N/(kg/s)")
    print(f"  Thrust Specific Fuel Consumption (TSFC): {SPECIFIC_FUEL_CONSUMPTION*1e6:.2f} mg/(N·s) or {SPECIFIC_FUEL_CONSUMPTION*3600:.4f} kg/(kN·hr)")

    return NET_THRUST_PER_UNIT_MASS_FLOW, SPECIFIC_FUEL_CONSUMPTION

# --- MAIN USER INTERFACE ---

def GET_FLOAT_INPUT(PROMPT, MIN_VAL=0.0, MAX_VAL=float('inf')):
    """Helper function to get validated float input."""
    while True:
        try:
            VALUE = float(input(PROMPT))
            if MIN_VAL <= VALUE <= MAX_VAL:
                return VALUE
            else:
                print(f"Input out of range. Please enter a value between {MIN_VAL} and {MAX_VAL}.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def RUN_ENGINE_ESTIMATOR():
    """Main function to run the interactive engine performance estimator."""
    print("Welcome to the Jet Engine Performance Estimator (Ideal Cycle Analysis)")
    print("--------------------------------------------------------------------")

    while True:
        print("\nChoose an engine type:")
        print("1. Turbojet Engine")
        print("2. Turbofan Engine (High and Low Bypass)")
        print("3. Turboprop Engine")
        print("4. Ramjet Engine")
        print("5. Exit")

        CHOICE = input("Enter your choice (1, 2, 3, 4, or 5): ")

        if CHOICE == '1':
            FLIGHT_MACH_NUMBER = GET_FLOAT_INPUT("Enter Flight Mach Number (e.g., 0.8): ", MIN_VAL=0.0)
            AMBIENT_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Ambient Temperature (K, e.g., 288.15): ", MIN_VAL=100.0)
            AMBIENT_PRESSURE_PA = GET_FLOAT_INPUT("Enter Ambient Pressure (Pa, e.g., 101325): ", MIN_VAL=10000.0)
            COMPRESSOR_PRESSURE_RATIO = GET_FLOAT_INPUT("Enter Compressor Pressure Ratio (e.g., 15): ", MIN_VAL=1.0)
            TURBINE_INLET_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Turbine Inlet Temperature (K, e.g., 1500): ", MIN_VAL=500.0)

            DIFFUSER_EFF = GET_FLOAT_INPUT("Enter Diffuser Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            COMPRESSOR_EFF = GET_FLOAT_INPUT("Enter Compressor Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            TURBINE_EFF = GET_FLOAT_INPUT("Enter Turbine Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            NOZZLE_EFF = GET_FLOAT_INPUT("Enter Nozzle Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)

            ESTIMATE_TURBOJET_PERFORMANCE(
                FLIGHT_MACH_NUMBER,
                AMBIENT_TEMPERATURE_K,
                AMBIENT_PRESSURE_PA,
                COMPRESSOR_PRESSURE_RATIO,
                TURBINE_INLET_TEMPERATURE_K,
                COMPRESSOR_ISENTROPIC_EFFICIENCY=COMPRESSOR_EFF,
                TURBINE_ISENTROPIC_EFFICIENCY=TURBINE_EFF,
                NOZZLE_ISENTROPIC_EFFICIENCY=NOZZLE_EFF,
                DIFFUSER_ISENTROPIC_EFFICIENCY=DIFFUSER_EFF
            )
        elif CHOICE == '2':
            FLIGHT_MACH_NUMBER = GET_FLOAT_INPUT("Enter Flight Mach Number (e.g., 0.8): ", MIN_VAL=0.0)
            AMBIENT_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Ambient Temperature (K, e.g., 288.15): ", MIN_VAL=100.0)
            AMBIENT_PRESSURE_PA = GET_FLOAT_INPUT("Enter Ambient Pressure (Pa, e.g., 101325): ", MIN_VAL=10000.0)
            COMPRESSOR_PRESSURE_RATIO = GET_FLOAT_INPUT("Enter Core Compressor Pressure Ratio (e.g., 10): ", MIN_VAL=1.0)
            FAN_PRESSURE_RATIO = GET_FLOAT_INPUT("Enter Fan Pressure Ratio (e.g., 1.5): ", MIN_VAL=1.0)
            BYPASS_RATIO = GET_FLOAT_INPUT("Enter Bypass Ratio (e.g., 5.0 for high bypass, 0.5 for low bypass): ", MIN_VAL=0.0)
            TURBINE_INLET_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Turbine Inlet Temperature (K, e.g., 1600): ", MIN_VAL=500.0)

            DIFFUSER_EFF = GET_FLOAT_INPUT("Enter Diffuser Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            FAN_EFF = GET_FLOAT_INPUT("Enter Fan Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            COMPRESSOR_EFF = GET_FLOAT_INPUT("Enter Core Compressor Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            TURBINE_EFF = GET_FLOAT_INPUT("Enter Turbine Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            NOZZLE_EFF = GET_FLOAT_INPUT("Enter Nozzle Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)

            ESTIMATE_TURBOFAN_PERFORMANCE(
                FLIGHT_MACH_NUMBER,
                AMBIENT_TEMPERATURE_K,
                AMBIENT_PRESSURE_PA,
                COMPRESSOR_PRESSURE_RATIO,
                FAN_PRESSURE_RATIO,
                BYPASS_RATIO,
                TURBINE_INLET_TEMPERATURE_K,
                COMPRESSOR_ISENTROPIC_EFFICIENCY=COMPRESSOR_EFF,
                TURBINE_ISENTROPIC_EFFICIENCY=TURBINE_EFF,
                FAN_ISENTROPIC_EFFICIENCY=FAN_EFF,
                NOZZLE_ISENTROPIC_EFFICIENCY=NOZZLE_EFF,
                DIFFUSER_ISENTROPIC_EFFICIENCY=DIFFUSER_EFF
            )
        elif CHOICE == '3':
            FLIGHT_MACH_NUMBER = GET_FLOAT_INPUT("Enter Flight Mach Number (e.g., 0.5): ", MIN_VAL=0.0)
            AMBIENT_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Ambient Temperature (K, e.g., 288.15): ", MIN_VAL=100.0)
            AMBIENT_PRESSURE_PA = GET_FLOAT_INPUT("Enter Ambient Pressure (Pa, e.g., 101325): ", MIN_VAL=10000.0)
            COMPRESSOR_PRESSURE_RATIO = GET_FLOAT_INPUT("Enter Compressor Pressure Ratio (e.g., 8): ", MIN_VAL=1.0)
            TURBINE_INLET_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Turbine Inlet Temperature (K, e.g., 1400): ", MIN_VAL=500.0)
            PROPELLER_EFFICIENCY = GET_FLOAT_INPUT("Enter Propeller Efficiency (0.0-1.0, e.g., 0.8): ", MIN_VAL=0.0, MAX_VAL=1.0)

            DIFFUSER_EFF = GET_FLOAT_INPUT("Enter Diffuser Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            COMPRESSOR_EFF = GET_FLOAT_INPUT("Enter Compressor Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            TURBINE_EFF = GET_FLOAT_INPUT("Enter Turbine Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            NOZZLE_EFF = GET_FLOAT_INPUT("Enter Nozzle Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)

            ESTIMATE_TURBOPROP_PERFORMANCE(
                FLIGHT_MACH_NUMBER,
                AMBIENT_TEMPERATURE_K,
                AMBIENT_PRESSURE_PA,
                COMPRESSOR_PRESSURE_RATIO,
                TURBINE_INLET_TEMPERATURE_K,
                PROPELLER_EFFICIENCY,
                COMPRESSOR_ISENTROPIC_EFFICIENCY=COMPRESSOR_EFF,
                TURBINE_ISENTROPIC_EFFICIENCY=TURBINE_EFF,
                NOZZLE_ISENTROPIC_EFFICIENCY=NOZZLE_EFF,
                DIFFUSER_ISENTROPIC_EFFICIENCY=DIFFUSER_EFF
            )
        elif CHOICE == '4':
            FLIGHT_MACH_NUMBER = GET_FLOAT_INPUT("Enter Flight Mach Number (e.g., 2.0 - Ramjets are for supersonic flight): ", MIN_VAL=0.0)
            AMBIENT_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Ambient Temperature (K, e.g., 216.65 for stratosphere): ", MIN_VAL=100.0)
            AMBIENT_PRESSURE_PA = GET_FLOAT_INPUT("Enter Ambient Pressure (Pa, e.g., 22632 for stratosphere): ", MIN_VAL=1000.0)
            COMBUSTOR_OUTLET_TOTAL_TEMPERATURE_K = GET_FLOAT_INPUT("Enter Combustor Outlet Total Temperature (K, e.g., 2000): ", MIN_VAL=500.0)

            DIFFUSER_EFF = GET_FLOAT_INPUT("Enter Diffuser Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)
            NOZZLE_EFF = GET_FLOAT_INPUT("Enter Nozzle Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): ", MIN_VAL=0.0, MAX_VAL=1.0)

            ESTIMATE_RAMJET_PERFORMANCE(
                FLIGHT_MACH_NUMBER,
                AMBIENT_TEMPERATURE_K,
                AMBIENT_PRESSURE_PA,
                COMBUSTOR_OUTLET_TOTAL_TEMPERATURE_K,
                DIFFUSER_ISENTROPIC_EFFICIENCY=DIFFUSER_EFF,
                NOZZLE_ISENTROPIC_EFFICIENCY=NOZZLE_EFF
            )
        elif CHOICE == '5':
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5.")

RUN_ENGINE_ESTIMATOR()

Welcome to the Jet Engine Performance Estimator (Ideal Cycle Analysis)
--------------------------------------------------------------------

Choose an engine type:
1. Turbojet Engine
2. Turbofan Engine (High and Low Bypass)
3. Turboprop Engine
4. Ramjet Engine
5. Exit
Enter your choice (1, 2, 3, 4, or 5): 1
Enter Flight Mach Number (e.g., 0.8): 0.75
Enter Ambient Temperature (K, e.g., 288.15): 258
Enter Ambient Pressure (Pa, e.g., 101325): 101325
Enter Compressor Pressure Ratio (e.g., 15): 14
Enter Turbine Inlet Temperature (K, e.g., 1500): 1400
Enter Diffuser Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): 1
Enter Compressor Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): 1
Enter Turbine Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): 1
Enter Nozzle Isentropic Efficiency (0.0-1.0, default 1.0 for ideal): 1

--- Turbojet Engine Analysis ---
Debug: TOTAL_TEMPERATURE_5 = 1076.9476820517793
Debug: STATIC_TEMPERATURE_9_ACTUAL = 592.0561039947664
  Flight Mach Number